# Merging Monthly County Unemployment Rates with SBA Loan Data

**Data Sources:**
- BLS LAUS county-level unemployment rates (1990-2014)
- Census Bureau ZCTA-to-County crosswalk (2010)
- SBA loan dataset (`sba_loan_cleaned.csv`)

**Steps:**
1. Build ZIP-to-County FIPS mapping
2. Prepare SBA loan dataset (clean BorrZip, extract year/month)
3. Parse and merge LAUS data one file at a time
4. Validation
5. Export

In [22]:
import pandas as pd
import numpy as np
import os

DATA_DIR = 'laus_data'
SBA_PATH = '/Users/yangmar/Desktop/MS&E 246/Project/preprocessing/sba_loan_cleaned.csv'

## 1. Build ZIP-to-County FIPS Mapping

Using Census Bureau ZCTA-County relationship file. When a ZIP spans multiple counties, keep the one with the highest population percentage.

In [23]:
# Load Census ZCTA-County crosswalk
zcta = pd.read_csv(os.path.join(DATA_DIR, 'zcta_county_rel_10.txt'), dtype=str)
print(f'Raw crosswalk rows: {len(zcta)}')
print(zcta.columns.tolist())
zcta.head()

Raw crosswalk rows: 44410
['ZCTA5', 'STATE', 'COUNTY', 'GEOID', 'POPPT', 'HUPT', 'AREAPT', 'AREALANDPT', 'ZPOP', 'ZHU', 'ZAREA', 'ZAREALAND', 'COPOP', 'COHU', 'COAREA', 'COAREALAND', 'ZPOPPCT', 'ZHUPCT', 'ZAREAPCT', 'ZAREALANDPCT', 'COPOPPCT', 'COHUPCT', 'COAREAPCT', 'COAREALANDPCT']


,ZCTA5,STATE,COUNTY,GEOID,POPPT,HUPT,AREAPT,AREALANDPT,ZPOP,ZHU,...,COAREA,COAREALAND,ZPOPPCT,ZHUPCT,ZAREAPCT,ZAREALANDPCT,COPOPPCT,COHUPCT,COAREAPCT,COAREALANDPCT
0,00601,72,001,72001,18465,7695,165132671,164333375,18570,7744,...,173777444,172725651,99.43,99.37,98.61,98.6,94.77,94.71,95.03,95.14
1,00601,72,141,72141,105,49,2326414,2326414,18570,7744,...,298027589,294039825,0.57,0.63,1.39,1.4,0.32,0.35,0.78,0.79
2,00602,72,003,72003,41520,18073,83734431,79288158,41520,18073,...,117948080,79904246,100,100,100,100,98.95,98.99,70.99,99.23
3,00603,72,005,72005,54689,25653,82063867,81880442,54689,25653,...,195741178,94608641,100,100,100,100,89.73,90.23,41.92,86.55
4,00606,72,093,72093,6276,2740,94864349,94851862,6615,2877,...,94864349,94851862,94.88,95.24,86.56,86.56,100,100,100,100


In [24]:
# Build ZIP -> County FIPS mapping (keep county with highest population share)
zcta['ZPOPPCT'] = zcta['ZPOPPCT'].astype(float)
zcta['county_fips'] = zcta['STATE'] + zcta['COUNTY']  # 5-digit FIPS

# For each ZIP, keep the county with the highest population percentage
zip_to_county = (
    zcta.sort_values('ZPOPPCT', ascending=False)
    .drop_duplicates(subset='ZCTA5', keep='first')
    [['ZCTA5', 'county_fips']]
    .rename(columns={'ZCTA5': 'zip_code'})
    .reset_index(drop=True)
)

print(f'Unique ZIP codes in crosswalk: {len(zip_to_county)}')
zip_to_county.head()

Unique ZIP codes in crosswalk: 33120


,zip_code,county_fips
0,99929,02275
1,74061,40147
2,74046,40037
3,74048,40105
4,74052,40037


## 2. Prepare SBA Loan Dataset

In [25]:
# Load SBA data
sba = pd.read_csv(SBA_PATH, dtype={'BorrZip': str, 'CDC_Zip': str},
                   parse_dates=['ApprovalDate', 'ChargeOffDate'])
print(f'SBA rows: {len(sba)}')
sba.head()

SBA rows: 127475


/var/folders/y6/q6ynf7q94qv9p0n2wqdfnblc0000gn/T/ipykernel_41205/172658666.py:2: DtypeWarning: Columns (0: NaicsSector) have mixed types. Specify dtype option on import or set low_memory=False.
  sba = pd.read_csv(SBA_PATH, dtype={'BorrZip': str, 'CDC_Zip': str},


,BorrZip,CDC_Zip,ThirdPartyLender_City,ThirdPartyLender_State,ThirdPartyDollars,GrossApproval,ApprovalDate,ApprovalFiscalYear,DeliveryMethod,subpgmdesc,TermInMonths,ProjectCounty,ProjectState,BusinessType,LoanStatus,ChargeOffDate,GrossChargeOffAmount,HasThirdParty,NaicsSector
0,66106,65109,UNKNOWN,UNKNOWN,0,166000,1990-01-02,1990,504,Sec. 504 - Loan Guarantees - Private Sector Fi...,12,WYANDOTTE,KS,INDIVIDUAL,PIF,NaT,0,0,UNKNOWN
1,92507,92106,UNKNOWN,UNKNOWN,0,117000,1990-01-02,1990,504,Sec. 504 - Loan Guarantees - Private Sector Fi...,240,RIVERSIDE,CA,INDIVIDUAL,PIF,NaT,0,0,UNKNOWN
2,46628,46601,UNKNOWN,UNKNOWN,0,261000,1990-01-03,1990,504,Sec. 504 - Loan Guarantees - Private Sector Fi...,120,ST JOSEPH,IN,CORPORATION,PIF,NaT,0,0,UNKNOWN
3,62946,62704,UNKNOWN,UNKNOWN,0,262000,1990-01-03,1990,504,Sec. 504 - Loan Guarantees - Private Sector Fi...,240,SALINE,IL,CORPORATION,CHGOFF,2003-03-28,0,0,UNKNOWN
4,84106,84109,UNKNOWN,UNKNOWN,0,154000,1990-01-03,1990,504,Sec. 504 - Loan Guarantees - Private Sector Fi...,240,SALT LAKE,UT,CORPORATION,PIF,NaT,0,0,UNKNOWN


In [26]:
# Clean BorrZip: ensure 5-digit string (left-pad with zeros, truncate ZIP+4)
sba['BorrZip_clean'] = sba['BorrZip'].str.strip().str[:5].str.zfill(5)

# Extract calendar year and month from ApprovalDate
sba['approval_year'] = sba['ApprovalDate'].dt.year
sba['approval_month'] = sba['ApprovalDate'].dt.month

print(f'Year range: {sba["approval_year"].min()} - {sba["approval_year"].max()}')
print(f'Sample BorrZip_clean: {sba["BorrZip_clean"].head().tolist()}')

Year range: 1990 - 2014
Sample BorrZip_clean: ['66106', '92507', '46628', '62946', '84106']


In [27]:
# Join BorrZip -> county_fips
sba = sba.merge(zip_to_county, left_on='BorrZip_clean', right_on='zip_code', how='left')

matched_borr = sba['county_fips'].notna().sum()
print(f'BorrZip match: {matched_borr}/{len(sba)} ({matched_borr/len(sba)*100:.1f}%)')

# Fallback: use CDC_Zip for unmatched rows
unmatched = sba['county_fips'].isna()
sba['CDC_Zip_clean'] = sba['CDC_Zip'].str.strip().str[:5].str.zfill(5)
cdc_fips = sba.loc[unmatched, 'CDC_Zip_clean'].map(zip_to_county.set_index('zip_code')['county_fips'])
sba.loc[unmatched, 'county_fips'] = cdc_fips

matched_total = sba['county_fips'].notna().sum()
print(f'After CDC_Zip fallback: {matched_total}/{len(sba)} ({matched_total/len(sba)*100:.1f}%)')
print(f'Recovered by CDC_Zip: {matched_total - matched_borr} loans')
print(f'Still unmatched: {sba["county_fips"].isna().sum()} loans ({sba.loc[sba["county_fips"].isna(), "BorrZip_clean"].nunique()} unique ZIPs)')

BorrZip match: 124676/127475 (97.8%)
After CDC_Zip fallback: 127338/127475 (99.9%)
Recovered by CDC_Zip: 2662 loans
Still unmatched: 137 loans (101 unique ZIPs)


## 3. Parse and Merge LAUS Data (one file at a time)

Each LAUS file covers a 5-year range. Process sequentially to manage memory.

The `series_id` format is: `LA` (2) + seasonal adj (1) + area code (15) + measure code (2).
- Filter to county areas: area code starts with `CN`
- Filter to measure code `03` (unemployment rate)
- Extract 5-digit county FIPS from area code: `CN` + state (2) + county (3) + padding

In [28]:
def parse_laus_file(filepath):
    """Parse a single LAUS data file and return county-level monthly unemployment rates."""
    df = pd.read_csv(filepath, sep='\t', dtype=str)
    df.columns = df.columns.str.strip()
    df['series_id'] = df['series_id'].str.strip()
    df['value'] = df['value'].str.strip()
    df['period'] = df['period'].str.strip()
    
    # Parse series_id: LA(2) + seasonal(1) + area_code(15) + measure_code(2)
    df['area_code'] = df['series_id'].str[3:18]
    df['measure_code'] = df['series_id'].str[18:20]
    
    # Filter: county areas (CN*), unemployment rate (03), monthly only (M01-M12)
    mask = (
        df['area_code'].str.startswith('CN') &
        (df['measure_code'] == '03') &
        df['period'].str.match(r'^M(0[1-9]|1[0-2])$')
    )
    df = df[mask].copy()
    
    # Extract 5-digit county FIPS: CN + state(2) + county(3) + padding
    df['county_fips'] = df['area_code'].str[2:7]
    df['year'] = df['year'].astype(int)
    df['month'] = df['period'].str[1:].astype(int)
    df['unemployment_rate'] = pd.to_numeric(df['value'], errors='coerce')
    
    return df[['county_fips', 'year', 'month', 'unemployment_rate']]

# Test with one file
test = parse_laus_file(os.path.join(DATA_DIR, 'la.data.0.CurrentU90-94'))
print(f'Parsed rows: {len(test)}')
print(f'Year range: {test["year"].min()}-{test["year"].max()}')
print(f'Unique counties: {test["county_fips"].nunique()}')
test.head()

Parsed rows: 193032
Year range: 1990-1994
Unique counties: 3218


,county_fips,year,month,unemployment_rate
49036,01001,1990,1,6.5
49037,01001,1990,2,6.4
49038,01001,1990,3,5.6
49039,01001,1990,4,6.6
49040,01001,1990,5,6.1


In [29]:
# Process each LAUS file and merge with corresponding SBA loans
laus_files = [
    ('la.data.0.CurrentU90-94', 1990, 1994),
    ('la.data.0.CurrentU95-99', 1995, 1999),
    ('la.data.0.CurrentU00-04', 2000, 2004),
    ('la.data.0.CurrentU05-09', 2005, 2009),
    ('la.data.0.CurrentU10-14', 2010, 2014),
]

merged_parts = []

for filename, year_start, year_end in laus_files:
    print(f'\nProcessing {filename} ({year_start}-{year_end})...')
    
    # Parse LAUS file
    laus = parse_laus_file(os.path.join(DATA_DIR, filename))
    print(f'  LAUS records: {len(laus)}')
    
    # Get SBA loans in this year range
    sba_subset = sba[
        (sba['approval_year'] >= year_start) & 
        (sba['approval_year'] <= year_end)
    ].copy()
    print(f'  SBA loans in range: {len(sba_subset)}')
    
    # Merge on county_fips + year + month
    merged = sba_subset.merge(
        laus,
        left_on=['county_fips', 'approval_year', 'approval_month'],
        right_on=['county_fips', 'year', 'month'],
        how='left'
    )
    matched = merged['unemployment_rate'].notna().sum()
    print(f'  Matched: {matched}/{len(merged)} ({matched/len(merged)*100:.1f}%)')
    
    merged_parts.append(merged)
    del laus  # Free memory

# Combine all parts
sba_merged = pd.concat(merged_parts, ignore_index=True)

# Handle loans outside 1990-2014 (no LAUS data)
sba_outside = sba[
    (sba['approval_year'] < 1990) | (sba['approval_year'] > 2014)
].copy()
sba_outside['unemployment_rate'] = np.nan
sba_outside['year'] = np.nan
sba_outside['month'] = np.nan

if len(sba_outside) > 0:
    print(f'\nLoans outside 1990-2014: {len(sba_outside)}')
    sba_merged = pd.concat([sba_merged, sba_outside], ignore_index=True)

print(f'\nFinal merged dataset: {len(sba_merged)} rows')


Processing la.data.0.CurrentU90-94 (1990-1994)...
  LAUS records: 193032
  SBA loans in range: 10504
  Matched: 10447/10504 (99.5%)

Processing la.data.0.CurrentU95-99 (1995-1999)...
  LAUS records: 193080
  SBA loans in range: 21591
  Matched: 21453/21591 (99.4%)

Processing la.data.0.CurrentU00-04 (2000-2004)...
  LAUS records: 193080
  SBA loans in range: 25980
  Matched: 25736/25980 (99.1%)

Processing la.data.0.CurrentU05-09 (2005-2009)...
  LAUS records: 193080
  SBA loans in range: 38238
  Matched: 37895/38238 (99.1%)

Processing la.data.0.CurrentU10-14 (2010-2014)...
  LAUS records: 193200
  SBA loans in range: 31162
  Matched: 30862/31162 (99.0%)

Final merged dataset: 127475 rows


## 4. Validation

In [30]:
# Match summary
total = len(sba_merged)
has_fips = sba_merged['county_fips'].notna().sum()
has_unemp = sba_merged['unemployment_rate'].notna().sum()

print(f'Total loans: {total}')
print(f'Matched to county FIPS via BorrZip: {has_fips} ({has_fips/total*100:.1f}%)')
print(f'Matched to monthly unemployment rate: {has_unemp} ({has_unemp/total*100:.1f}%)')
print(f'\nUnmatched ZIP codes (no county FIPS):')
unmatched_zips = sba_merged.loc[sba_merged['county_fips'].isna(), 'BorrZip_clean'].value_counts()
print(f'  {unmatched_zips.shape[0]} unique ZIPs')
if len(unmatched_zips) > 0:
    print(f'  Top 10: {unmatched_zips.head(10).to_dict()}')

print(f'\nLoans with FIPS but no unemployment rate:')
no_rate = sba_merged['county_fips'].notna() & sba_merged['unemployment_rate'].isna()
print(f'  {no_rate.sum()} loans')

Total loans: 127475
Matched to county FIPS via BorrZip: 127338 (99.9%)
Matched to monthly unemployment rate: 126393 (99.2%)

Unmatched ZIP codes (no county FIPS):
  101 unique ZIPs
  Top 10: {'28603': 6, '83303': 6, '59403': 4, '29731': 3, '59903': 3, '33846': 3, '78042': 3, '00922': 3, '59806': 2, '99999': 2}

Loans with FIPS but no unemployment rate:
  945 loans


In [31]:
# Spot check: LA County (FIPS 06037), October 2009 — should be ~12-13%
la_oct_2009 = sba_merged[
    (sba_merged['county_fips'] == '06037') &
    (sba_merged['approval_year'] == 2009) &
    (sba_merged['approval_month'] == 10)
]
if len(la_oct_2009) > 0:
    rate = la_oct_2009['unemployment_rate'].iloc[0]
    print(f'LA County, Oct 2009 unemployment rate: {rate}%')
    print(f'Expected: ~12-13% — {"PASS" if 11 <= rate <= 14 else "CHECK"}')
else:
    # Check LAUS directly for this county-month
    laus_check = parse_laus_file(os.path.join(DATA_DIR, 'la.data.0.CurrentU05-09'))
    la_check = laus_check[(laus_check['county_fips'] == '06037') & 
                          (laus_check['year'] == 2009) & 
                          (laus_check['month'] == 10)]
    print(f'No SBA loans for LA County Oct 2009, but LAUS rate: {la_check["unemployment_rate"].values}')

LA County, Oct 2009 unemployment rate: 12.3%
Expected: ~12-13% — PASS


## 5. Export

In [32]:
# Clean up merge columns
sba_final = sba_merged.drop(columns=['zip_code', 'year', 'month'], errors='ignore')

# Rename for clarity
sba_final = sba_final.rename(columns={'unemployment_rate': 'county_unemployment_rate'})

print(f'Final columns: {sba_final.columns.tolist()}')
print(f'Shape: {sba_final.shape}')
print(f'\ncounty_unemployment_rate stats:')
print(sba_final['county_unemployment_rate'].describe())

Final columns: ['BorrZip', 'CDC_Zip', 'ThirdPartyLender_City', 'ThirdPartyLender_State', 'ThirdPartyDollars', 'GrossApproval', 'ApprovalDate', 'ApprovalFiscalYear', 'DeliveryMethod', 'subpgmdesc', 'TermInMonths', 'ProjectCounty', 'ProjectState', 'BusinessType', 'LoanStatus', 'ChargeOffDate', 'GrossChargeOffAmount', 'HasThirdParty', 'NaicsSector', 'BorrZip_clean', 'approval_year', 'approval_month', 'county_fips', 'CDC_Zip_clean', 'county_unemployment_rate']
Shape: (127475, 25)

county_unemployment_rate stats:
count    126393.000000
mean          6.031359
std           2.781826
min           0.700000
25%           4.100000
50%           5.300000
75%           7.500000
max          38.400000
Name: county_unemployment_rate, dtype: float64


In [33]:
# Save final merged dataset
sba_final.to_csv('../sba_loan_with_unemployment.csv', index=False)
print(f'Saved sba_loan_with_unemployment.csv ({len(sba_final)} rows)')

# Save ZIP-to-County mapping for reuse
zip_to_county.to_csv('zip_to_county_fips.csv', index=False)
print(f'Saved zip_to_county_fips.csv ({len(zip_to_county)} rows)')

Saved sba_loan_with_unemployment.csv (127475 rows)
Saved zip_to_county_fips.csv (33120 rows)


In [34]:
sba_final.isnull().sum()

BorrZip                          0
CDC_Zip                          0
ThirdPartyLender_City            0
ThirdPartyLender_State           0
ThirdPartyDollars                0
GrossApproval                    0
ApprovalDate                     0
ApprovalFiscalYear               0
DeliveryMethod                   0
subpgmdesc                       0
TermInMonths                     0
ProjectCounty                    0
ProjectState                     0
BusinessType                     0
LoanStatus                       0
ChargeOffDate               118510
GrossChargeOffAmount             0
HasThirdParty                    0
NaicsSector                      0
BorrZip_clean                    0
approval_year                    0
approval_month                   0
county_fips                    137
CDC_Zip_clean                    0
county_unemployment_rate      1082
dtype: int64

In [35]:
sba_final[sba_final['county_fips'].isnull()]

,BorrZip,CDC_Zip,ThirdPartyLender_City,ThirdPartyLender_State,ThirdPartyDollars,GrossApproval,ApprovalDate,ApprovalFiscalYear,DeliveryMethod,subpgmdesc,...,ChargeOffDate,GrossChargeOffAmount,HasThirdParty,NaicsSector,BorrZip_clean,approval_year,approval_month,county_fips,CDC_Zip_clean,county_unemployment_rate
115,31717,0,UNKNOWN,UNKNOWN,0,62000,1990-02-06,1990,504,Sec. 504 - Loan Guarantees - Private Sector Fi...,...,NaT,0,0,81,31717,1990,2,NaN,00000,NaN
293,28603,28603,UNKNOWN,UNKNOWN,0,353000,1990-03-28,1990,504,Sec. 504 - Loan Guarantees - Private Sector Fi...,...,NaT,0,0,31,28603,1990,3,NaN,28603,NaN
866,71161,71133,UNKNOWN,UNKNOWN,0,750000,1990-08-24,1990,504,Sec. 504 - Loan Guarantees - Private Sector Fi...,...,NaT,0,0,42,71161,1990,8,NaN,71133,NaN
1244,28603,28603,UNKNOWN,UNKNOWN,0,750000,1990-12-10,1991,504,Sec. 504 - Loan Guarantees - Private Sector Fi...,...,NaT,0,0,UNKNOWN,28603,1990,12,NaN,28603,NaN
1482,658,0,UNKNOWN,UNKNOWN,0,239000,1991-02-26,1991,504,Sec. 504 - Loan Guarantees - Private Sector Fi...,...,NaT,0,0,33,00658,1991,2,NaN,00000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116032,84407,83303,WALLA WALLA,WA,2642674,1902000,2012-07-27,2012,504,Sec. 504 - Loan Guarantees - Private Sector Fi...,...,NaT,0,1,81,84407,2012,7,NaN,83303,NaN
117232,32716,32315,COLUMBUS,OH,358900,303000,2012-09-06,2012,504,Sec. 504 - Loan Guarantees - Private Sector Fi...,...,NaT,0,1,42,32716,2012,9,NaN,32315,NaN
118074,17534,17608,LEMOYNE,PA,4173000,3237000,2012-09-28,2012,504,Sec. 504 - Loan Guarantees - Private Sector Fi...,...,NaT,0,1,72,17534,2012,9,NaN,17608,NaN
118423,32772,32315,WILMINGTON,DE,537500,445000,2012-10-17,2013,504,Sec. 504 - Loan Guarantees - Private Sector Fi...,...,NaT,0,1,23,32772,2012,10,NaN,32315,NaN
